<a href="https://colab.research.google.com/github/amrutha1810/ML-LAB/blob/main/1BM22CS036_Lab_11_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 1: Load the digits dataset
digits = load_digits()
X = digits.data
y = digits.target

# Step 2: Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Step 3: Perform feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Apply PCA with 2 components
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Step 5: Train logistic regression on PCA-reduced data
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_pca, y_train)

# Step 6: Predict and evaluate
y_pred = log_reg.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy using PCA with 2 components: {accuracy:.4f}')


Accuracy using PCA with 2 components: 0.5694


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from scipy import stats

# Step 1: Load the dataset
df = pd.read_csv('heart.csv')

# Step 2: Remove outliers using Z-score
df_numeric = df.select_dtypes(include=[np.number])
z_scores = np.abs(stats.zscore(df_numeric))
df_no_outliers = df[(z_scores < 3).all(axis=1)]

# Step 3: Convert text columns to numbers
df_processed = df_no_outliers.copy()
for col in df_processed.select_dtypes(include=['object']).columns:
    if df_processed[col].nunique() == 2:
        le = LabelEncoder()
        df_processed[col] = le.fit_transform(df_processed[col])
    else:
        df_processed = pd.get_dummies(df_processed, columns=[col])

# Separate features and target
X = df_processed.drop('HeartDisease', axis=1)
y = df_processed['HeartDisease']

# Step 4: Apply scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Train models and compare accuracies
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier()
}

print("Accuracy without PCA:")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: {accuracy_score(y_test, y_pred):.4f}")

# Step 6: Use PCA for dimensionality reduction
pca = PCA(n_components=5)  # You can tune this value
X_pca = pca.fit_transform(X_scaled)

X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

print("\nAccuracy with PCA:")
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred_pca = model.predict(X_test_pca)
    print(f"{name}: {accuracy_score(y_test, y_pred_pca):.4f}")


Accuracy without PCA:
Logistic Regression: 0.8833
SVM: 0.8778
Random Forest: 0.8889

Accuracy with PCA:
Logistic Regression: 0.8667
SVM: 0.8556
Random Forest: 0.8722
